# German Textsummary CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time

## Params

In [2]:
BATCH_SIZE = 10

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [3]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

In [4]:
en_de_prefix = tf.reshape(tokenizer.encode("summarize: en_to_ger ", return_tensors="tf"), (-1,))
de_en_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_en ", return_tensors="tf"), (-1,))
en_en_prefix = tf.reshape(tokenizer.encode("summarize: en_to_en ", return_tensors="tf"), (-1,))
de_de_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_ger ", return_tensors="tf"), (-1,))

In [5]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275392 
Total params: 222,903,936
Trainable params: 222,903,936
Non-trainable params: 0
_________________________________________________________________


In [7]:
ckpt_file = "../models/ckpt/checkpoint_cl1.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [8]:
import numpy as np
MAX_ARTICLE_LEN = 512
MAX_HIGHLIGHT_LEN = 150
GLOBAL_BATCH_SIZE = 8

def get_tfrecord_dataset(file_name):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
        'ger_y_labels': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
        'en_y_labels': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
    }

    dataset = tf.data.TFRecordDataset(f"../data/{file_name}.tfrecord")

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        i  = tf.random.uniform((1,),0,4,dtype=tf.int32)[0]
        if i == 0:
            return tf.concat([de_de_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'], record['ger_y_labels']
        elif i == 1:
            return tf.concat([en_de_prefix, record['en_x']], axis=0), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['ger_y'], record['ger_y_ids'], record['ger_y_labels']
        elif i == 2:
            return tf.concat([de_en_prefix, record['ger_x']], axis=0), tf.concat([tf.ones(8, dtype=tf.int32), record['ger_x_mask']], axis=0), record['en_y'], record['en_y_ids'], record['en_y_labels']
        else:
            return tf.concat([en_en_prefix, record['en_x']], axis=0), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids'], record['en_y_labels']
 
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(100)
    return dataset.batch(GLOBAL_BATCH_SIZE)

test_ds = get_tfrecord_dataset("corss_lingual_test_cnn_daily_mail")

In [9]:
for i in test_ds.take(1):
    summaries = i[0]
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    for j in pred:
        print("----------")
        print(j)

----------
summarize: ger_to_en (CNN) Fünf Kämpfer der Arbeiterpartei Kurdistans wurden bei Zusammenstößen mit türkischen Streitkräften in der Osttürkei getötet und ein weiterer verletzt, wie das Militär des Landes am Samstag mitteilte. Auch vier türkische Soldaten seien bei den Kämpfen in der östlichen Stadt Agri verletzt worden, teilten die Streitkräfte in einer schriftlichen Erklärung mit. Die kurdischen Separatisten eröffneten aus großer Entfernung das Feuer auf türkische Soldaten, die ein Gebiet in Agri vor einem Frühlingsfest sicherten, berichtete CNN Türk. Der türkische Ministerpräsident Ahmet Davutoglu verurteilte die Gewalt und teilte via Twitter mit, dass "die türkische Armee die angemessene Antwort auf den abscheulichen Angriff in Agri gibt". Auch der türkische Präsident Recep Tayyip Erdogan verurteilte den Anschlag scharf und bezeichnete ihn als Versuch der kurdischen Separatisten, "in den Lösungsprozess (mit den Kurden) in unserem Land einzugreifen". Letzten Monat forderte

## Evaluation
### Define Rouge Score

In [10]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Rouge Score

In [20]:
predictions = []
rouge_score = RougeScore()
start_time = time.time()

for i, (input_ids, input_mask, y, y_ids, y_labels) in enumerate(test_ds):   
    summaries = model.generate(
        input_ids=input_ids, 
        attention_mask=input_mask, 
        early_stopping=True, 
        max_length=150, 
        length_penalty=100.0,
#         min_length=40, 
#         do_sample= True, 
        repetition_penalty=2.0,
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1
    )

    articles = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]
    
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
    for pred_sent, real_sent, article_sent in zip(pred, real, articles):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 3:
        # otherwise it will take ages
        break


# rouge_score.result()

0 : time genreate batch: 2.753051686286926


### Lets have a look at some of these predicted summaries

In [21]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [22]:
print(get_random_prediction())

article: summarize: ger_to_en Normale britische Tennisfans stehen in diesem Sommer vor einem Gerangel wie in Wimbledon um Davis-Cup-Viertelfinaltickets und dürften am Ende weniger als die Hälfte der Gesamtsumme erhalten. Für das Spiel gegen Frankreich im Juli im Londoner Queen 's Club wird eine Kapazität von 7.000 Plätzen zur Verfügung stehen, aber für die Art von Fans, die im letzten Monat in Glasgow für eine derart elektrisierende Atmosphäre gesorgt haben, könnten deutlich unter 3.000 Tickets zur Verfügung stehen. Der Rest geht an verschiedene Organisationen und Leitungsgremien, wobei ein beträchtlicher Prozentsatz an die gut betuchten Mitglieder der Queen geht, die im Rahmen des Deals dort das Sagen haben. Britische Tennisfans stehen in diesem Sommer vor einem Gerangel um Davis-Cup-Viertelfinaltickets wie in Wimbledon. Andy Murray genießt an der Seite seines Freundes Ross Hutchins im Camp Nou einen Platz in der Champions League. Die Lawn Tennis Association kam dem Wunsch von Andy Mu

In [23]:
print(get_random_prediction())

article: summarize: ger_to_ger West Ham diskutiert einen Deal für das jamaikanische Starlet DeShane Beckford, nachdem er vor Gericht überzeugt hatte. Der geschickte 17-jährige Stürmer von Montego Bay United wurde Anfang des Monats zum Training in die Akademie von West Ham eingeladen und hat die Trainer nach zwei Wochen beim Club beeindruckt. Beckford hat auch Angebote von Vereinen aus Belgien. Beckford, der mit einer Vielzahl europäischer Clubs in Verbindung gebracht wird, gilt als einer der heißesten Kandidaten für den Aufstieg aus der Karibik. Premier League Outfit West Ham nähert sich der Unterschrift des jamaikanischen Starlets DeShane Beckford. Unterdessen hat West Ham enthüllt, dass Dauerkarten im Olympiastadion nur £289 kosten werden. Die Hammers werden die günstigste Preisstrategie in der Barclays Premier League haben, um das 54.000 Zuschauer fassende Stadion zu füllen, wenn sie den Wechsel für die Saison 2016 / 17 vornehmen.

pred sentence: toffees nähern sich der Unterschrift

In [24]:
print(get_random_prediction())

article: summarize: en_to_ger Paris Saint-Germain face Nice on Saturday, hoping to take Ligue 1s top spot from Lyon but do so with a host of key stars missing, including captain Thiago Silva who is recuperating at home from a thigh injury. Zlatan Ibrahimovic, Marco Verratti and Thiago Motta all join Silva on the sidelines for the trip to the Mediterranean coast, while David Luiz is still not fully fit as he recovers from a thigh problem, although he is still set to start. Silva was pictured nursing his problem at home ahead of the game, with a full diagnosis yet to be completed by PSG's medical team. Thiago Silva who is recuperating at home from a thigh injury and will miss Paris Saint-Germain's game at Nice . Silva was substituted in the defeat by Barcelona in midweek and the injury is still keeping him out . 'Game Ready', as seen being used by Thiago Silva, is a cold therapy compression system used to treat sports injuries. It circulates cold water around the injury, cooling it down 

In [25]:
print(get_random_prediction())

article: summarize: en_to_en At least 34 people were arrested after hundreds of protesters gathered in New York City on Tuesday night to march against police brutality. The march was organized by National Actions to Stop Murder By Police. Many of the protesters cited the deaths of Eric Garner in Staten Island and Walter Scott in South Carolina. The protesters marched from Manhattans Union Square and across the Brooklyn Bridge where they partially blocked traffic. Scroll down for video . Hundreds of protesters gathered in New York City on Tuesday night to march against police brutality . Demonstrators climb on the Brooklyn Bridge during the protest against police brutality . The march was organized by National Actions to Stop Murder By Police . Stephen Davis, the Police Department's chief spokesman, said 34 people had been arrested by early evening . Many of the protesters cited the deaths of Eric Garner in Staten Island and Walter Scott in South Carolina . Organizers say Tuesday's prot

In [26]:
print(get_random_prediction())

article: summarize: ger_to_en Dougie Freedman steht kurz davor, sich auf einen neuen Zweijahresvertrag bei Nottingham Forest zu einigen. Freedman hat Forest stabilisiert, seit er Kultheld Stuart Pearce abgelöst hat, und die Clubbesitzer sind zufrieden mit seiner Arbeit am City Ground. Dougie Freedman wird einen neuen Vertrag bei Nottingham Forest unterzeichnen. Freedman hat auf dem City Ground beeindruckt, seit er Stuart Pearce im Februar ersetzt hat. Sie haben einen kühnen Versuch auf den Play-off-Plätzen unternommen, als Freedman Pearce ersetzte, aber in den letzten Wochen abgesprungen ist. Das hat Forest 's Eigentum nicht daran gehindert, Schritte zu unternehmen, um Freedman einen Vertrag für die nächsten beiden Spielzeiten zu sichern.

pred sentence: <extra_id_0>FULL: Dougie Freedman is set to sign a new contract at Nottingham Forest . Stuart Pearcen has been sacked by the club since February - and the owners are happy with his work he has done at the City Ground if he stays on . F

In [27]:
for i in range(10):
    print(get_random_prediction())

article: summarize: ger_to_en Normale britische Tennisfans stehen in diesem Sommer vor einem Gerangel wie in Wimbledon um Davis-Cup-Viertelfinaltickets und dürften am Ende weniger als die Hälfte der Gesamtsumme erhalten. Für das Spiel gegen Frankreich im Juli im Londoner Queen 's Club wird eine Kapazität von 7.000 Plätzen zur Verfügung stehen, aber für die Art von Fans, die im letzten Monat in Glasgow für eine derart elektrisierende Atmosphäre gesorgt haben, könnten deutlich unter 3.000 Tickets zur Verfügung stehen. Der Rest geht an verschiedene Organisationen und Leitungsgremien, wobei ein beträchtlicher Prozentsatz an die gut betuchten Mitglieder der Queen geht, die im Rahmen des Deals dort das Sagen haben. Britische Tennisfans stehen in diesem Sommer vor einem Gerangel um Davis-Cup-Viertelfinaltickets wie in Wimbledon. Andy Murray genießt an der Seite seines Freundes Ross Hutchins im Camp Nou einen Platz in der Champions League. Die Lawn Tennis Association kam dem Wunsch von Andy Mu

## Downsides
The start token makes usually no sense.

In [14]:
result_path = "../data/t5base_result_german.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")